# scRFEjun19

In [1]:
# AnnDatasubset
# Angela sent to Madeline on June 22

In [2]:
# Imports 
import numpy as np
import pandas as pd
import scanpy as sc
import random
from anndata import read_h5ad
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
import seaborn as sns
import matplotlib.pyplot as plt
import scanpy.external as sce
import logging as logg

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the multiarray numpy extension module failed.  Most
likely you are trying to import a failed build of numpy.
Here is how to proceed:
- If you're working with a numpy git repository, try `git clean -xdf`
  (removes all files not under version control) and rebuild numpy.
- If you are simply trying to use the numpy version that you have installed:
  your installation is broken - please reinstall numpy.
- If you have already reinstalled and that did not fix the problem, then:
  1. Check that you are using the Python you expect (you're using /Users/madelinepark/anaconda3/envs/scanpytest/bin/python),
     and that you have no directories in your PATH or PYTHONPATH that can
     interfere with the Python and numpy versions you're trying to use.
  2. If (1) looks fine, you can open a new issue at
     https://github.com/numpy/numpy/issues.  Please include details on:
     - how you installed Python
   

In [3]:
mouse_tfs = pd.read_csv("/home/angela/maca/GO_term_summary_20171110_222852.csv")
mouse_tfs.head()

,MGI Gene/Marker ID,Symbol,Name,Chr,Qualifier,Annotated Term,Context,Proteoform,Evidence,Inferred From,Reference(s)
0,MGI:1919321,2010315B03Rik,RIKEN cDNA 2010315B03 gene,9,NaN,DNA binding transcription factor activity,NaN,NaN,IBA,PTN001225609,J:161428
1,MGI:1924053,2410141K09Rik,RIKEN cDNA 2410141K09 gene,13,NaN,DNA binding transcription factor activity,NaN,NaN,IBA,PTN001225609,J:161428
2,MGI:1922662,2610021A01Rik,RIKEN cDNA 2610021A01 gene,7,NaN,DNA binding transcription factor activity,NaN,NaN,IBA,PTN001225609,J:161428
3,MGI:1919389,2610044O15Rik8,RIKEN cDNA 2610044O15 gene,8,NaN,DNA binding transcription factor activity,NaN,NaN,IBA,PTN001225609,J:161428
4,MGI:3045322,4932411N23Rik,RIKEN cDNA 4932411N23 gene,X,NaN,RNA polymerase II transcription factor activit...,NaN,NaN,IBA,PTN000067606,J:161428


In [32]:
adata = read_h5ad(
    "/home/angela/maca/tabula-muris-senis/0_data_ingest/01_figure_1/tabula-muris-senis-droplet-processed-official-annotations.h5ad")
adata

AnnData object with n_obs × n_vars = 245389 × 20138 
    obs: 'age', 'cell', 'cell_ontology_class', 'cell_ontology_id', 'free_annotation', 'method', 'mouse.id', 'n_genes', 'sex', 'subtissue', 'tissue', 'tissue_free_annotation', 'n_counts', 'louvain', 'leiden'
    var: 'n_cells', 'means', 'dispersions', 'dispersions_norm', 'highly_variable'
    uns: 'age_colors', 'leiden', 'louvain', 'louvain_colors', 'neighbors', 'pca', 'tissue_colors'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'

In [33]:
adata = adata[:,adata.var_names[adata.var_names.isin(mouse_tfs['Symbol'])]]
adata

View of AnnData object with n_obs × n_vars = 245389 × 1044 
    obs: 'age', 'cell', 'cell_ontology_class', 'cell_ontology_id', 'free_annotation', 'method', 'mouse.id', 'n_genes', 'sex', 'subtissue', 'tissue', 'tissue_free_annotation', 'n_counts', 'louvain', 'leiden'
    var: 'n_cells', 'means', 'dispersions', 'dispersions_norm', 'highly_variable'
    uns: 'age_colors', 'leiden', 'louvain', 'louvain_colors', 'neighbors', 'pca', 'tissue_colors'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'

In [ ]:
# adataLiver = read_h5ad('/Users/madelinepark/Downloads/Liver_droplet.h5ad')

adataLiver = adata[adata.obs['tissue']=='Liver'].copy()
adataLiver

In [22]:
def remove_cats(adata):

    cat_columns = adata.obs.select_dtypes(['category']).columns
    adata.obs[cat_columns] = adata.obs[cat_columns].astype(str)
    
    return adata

In [23]:
def filterNormalize (dataMatrix, classOfInterest):
    np.random.seed(644685)
#     sc.logging.print_versions()
#     sc.settings.verbosity = 3      
#     sc.logging.print_versions()
#     dataMatrix.obs['n_counts'] = dataMatrix.X.sum(axis=1).A1
    sc.pp.filter_cells(dataMatrix, min_genes=0)
    sc.pp.filter_genes(dataMatrix, min_cells=0)
#     dataMatrix = dataMatrix[dataMatrix.obs['n_counts'] > 1500, :]
#     sc.pp.normalize_per_cell(dataMatrix, counts_per_cell_after=1e5)
#     sc.pp.log1p(dataMatrix)
#     dataMatrix.raw = dataMatrix
    dataMatrix = dataMatrix[dataMatrix.obs[classOfInterest]!='nan']
    dataMatrix = dataMatrix[~dataMatrix.obs[classOfInterest].isna()]
    
    return dataMatrix

In [24]:
def labelSplit (dataMatrix, classOfInterest, labelOfInterest):
    dataMatrix = filterNormalize (dataMatrix, classOfInterest)
    dataMatrix.obs['classification_group'] = 'B'
    dataMatrix.obs.loc[dataMatrix.obs[dataMatrix.obs[classOfInterest]==labelOfInterest]
                   .index,'classification_group'] = 'A'
    return dataMatrix

In [25]:
def downsampleToSmallestCategory(dataMatrix, 
        classOfInterest = 'classification_group',
        random_state = None,
        min_cells = 15,
        keep_small_categories = True
) -> sc.AnnData:
    """
    returns an annData object in which all categories in 'classOfInterest' have
    the same size
    classOfInterest
        column with the categories to downsample
    min_cells
        Minimum number of cells to downsample.
        Categories having less than `min_cells` are discarded unless
        keep_small_categories is True
    keep_small_categories
        Be default categories with less than min_cells are discarded.
        Set to true to keep them
    """

    counts = dataMatrix.obs[classOfInterest].value_counts(sort=False)
    if len(counts[counts < min_cells]) > 0 and keep_small_categories is False:
        logg.warning(
            "The following categories have less than {} cells and will be "
            "ignored: {}".format(min_cells, dict(counts[counts < min_cells]))
        )
    min_size = min(counts[counts >= min_cells])
    sample_selection = None
    for sample, num_cells in counts.items():
        if num_cells <= min_cells:
            if keep_small_categories:
                sel = dataMatrix.obs.index.isin(
                    dataMatrix.obs[dataMatrix.obs[classOfInterest] == sample].index)
            else:
                continue
        else:
            sel = dataMatrix.obs.index.isin(
                dataMatrix.obs[dataMatrix.obs[classOfInterest] == sample]
                .sample(min_size, random_state=random_state)
                .index
            )
        if sample_selection is None:
            sample_selection = sel
        else:
            sample_selection |= sel
    logg.info(
        "The cells in category {!r} had been down-sampled to have each {} cells. "
        "The original counts where {}".format(classOfInterest, min_size, dict(counts))
    )
    return dataMatrix[sample_selection].copy()

In [26]:
def makeOneForest (dataMatrix, classOfInterest, labelOfInterest, nEstimators = 5000, 
                   randomState = 0,  nJobs = -1, oobScore = True, Step = 0.2, Cv = 5): 
    """
    Builds and runs a random forest for one label in a class of interest
    
    Parameters
    ----------
    dataMatrix : anndata object
        The data file of interest
    classOfInterest : str
        The class you will split the data by in the set of dataMatrix.obs
    labelOfInterest : str
        The specific label within the class that the random forezt will run a 
        "one vs all" classification on
    nEstimators : int
        The number of trees in the forest
    randomState : int
        Controls random number being used
    nJobs : int
        The number of jobs to run in parallel
    oobScore : bool
        Whether to use out-of-bag samples to estimate the generalization accuracy
    Step : float
        Corresponds to percentage of features to remove at each iteration
    Cv : int
        Determines the cross-validation splitting strategy
        
    Returns
    -------
    feature_selected : list
        list of top features from random forest
    selector.estimator_.feature_importances_ : list
        list of top ginis corresponding to to features
    
    """
    splitDataMatrix = labelSplit (dataMatrix, classOfInterest, labelOfInterest)
    downsampledMatrix = downsampleToSmallestCategory (dataMatrix = splitDataMatrix, 
        classOfInterest = 'classification_group',
        random_state = None, min_cells = 15, keep_small_categories = False)
    print('downsampled Matrix')
#     print("labelOfInterest")

    print(pd.DataFrame(downsampledMatrix.obs.groupby(['classification_group',classOfInterest])[classOfInterest].count()))
#     display (downsampledMatrix.obs['classification_group'])
#     print(set(downsampledMatrix.obs['classification_group']))
    
    feat_labels = downsampledMatrix.var_names 
    X = downsampledMatrix.X
    y = downsampledMatrix.obs['classification_group']
    
    clf = RandomForestClassifier(n_estimators = nEstimators, random_state = randomState, 
                                 n_jobs = nJobs, oob_score = oobScore)
    selector = RFECV(clf, step = Step, cv = Cv)
    
    clf.fit(X, y)
    selector.fit(X, y)
    feature_selected = feat_labels[selector.support_] 
#     display(downsampledMatrix.obs)

    dataMatrix.obs['classification_group'] = 'B'
#     print('corresponding')
#     display(downsampledMatrix.obs)

    return feature_selected, selector.estimator_.feature_importances_

In [27]:
def resultWrite (classOfInterest, results_df, labelOfInterest,
                feature_selected, feature_importance):

    column_headings = [] 
    column_headings.append(labelOfInterest)
    column_headings.append(labelOfInterest + '_gini')
    resaux = pd.DataFrame(columns = column_headings)
    resaux[labelOfInterest] = feature_selected
    resaux[labelOfInterest + '_gini'] = feature_importance
    resaux = resaux.sort_values(by = [labelOfInterest + '_gini'], ascending = False)
    resaux.reset_index(drop = True, inplace = True)

    results_df = pd.concat([results_df, resaux], axis=1)
    return results_df 

In [28]:
def scRFE(adata, classOfInterest, nEstimators = 5000, randomState = 0,  
                  nJobs = -1, oobScore = True, Step = 0.2, Cv = 5):
    """
    Builds and runs a random forest with one vs all classification for each label 
    for one class of interest
    
    Parameters
    ----------
    dataMatrix : anndata object
        The data file of interest
    classOfInterest : str
        The class you will split the data by in the set of dataMatrix.obs
    labelOfInterest : str
        The specific label within the class that the random forezt will run a 
        "one vs all" classification on
    nEstimators : int
        The number of trees in the forest
    randomState : int
        Controls random number being used
    nJobs : int
        The number of jobs to run in parallel
    oobScore : bool
        Whether to use out-of-bag samples to estimate the generalization accuracy
    Step : float
        Corresponds to percentage of features to remove at each iteration
    Cv : int
        Determines the cross-validation splitting strategy
        
    Returns
    -------
    results_df : pd.DataFrame
        Dataframe with results for each label in the class, formatted as 
        "label" for one column, then "label + gini" for the corresponding column
    
    """
    dataMatrix = adata.copy()
    dataMatrix = remove_cats(dataMatrix)
    
    print("Original dataset ",dataMatrix.shape)
    dataMatrix = filterNormalize (dataMatrix, classOfInterest)
    print("Filtered dataset ",dataMatrix.shape)
    print(pd.DataFrame(dataMatrix.obs.groupby([classOfInterest])[classOfInterest].count()))
    
    results_df = pd.DataFrame()
    for labelOfInterest in np.unique(dataMatrix.obs[classOfInterest]): 
        print(labelOfInterest)
        dataMatrix_labelOfInterest = dataMatrix.copy()
        feature_selected, feature_importance = makeOneForest(dataMatrix_labelOfInterest, 
                                                             classOfInterest, 
                          labelOfInterest = labelOfInterest)
    
        results_df = resultWrite (classOfInterest, results_df, 
                            labelOfInterest = labelOfInterest, 
                    feature_selected = feature_selected,  
                    feature_importance = feature_importance)
        
#     finaldf = makeOneForest (dataMatrix, classOfInterest, labelOfInterest = labelOfInterest)[2]

    return results_df

In [ ]:
senis_facs_tfs_age = scRFE (adata, classOfInterest = 'age', 
                       nEstimators = 10, randomState = 0,  
                  nJobs = -1, oobScore = True, Step = 0.2, Cv = 3)

Original dataset  (245389, 1044)
Filtered dataset  (245389, 1044)
       age
age       
18m  44645
1m   25980
21m  35828
24m  37660
30m  55674
3m   45602
18m


Trying to set attribute `.obs` of view, copying.


downsampled Matrix
                            age
classification_group age       
A                    18m  44645
B                    1m    5757
                     21m   8031
                     24m   8275
                     30m  12405
                     3m   10177
1m


Trying to set attribute `.obs` of view, copying.


downsampled Matrix
                            age
classification_group age       
A                    1m   25980
B                    18m   5233
                     21m   4160
                     24m   4517
                     30m   6584
                     3m    5486
21m


...

downsampled Matrix
                            age
classification_group age       
A                    24m  37660
B                    18m   8088
                     1m    4858
                     21m   6395
                     30m   9966
                     3m    8353
30m


Trying to set attribute `.obs` of view, copying.


downsampled Matrix
                            age
classification_group age       
A                    30m  55674
B                    18m  13159
                     1m    7713
                     21m  10388
                     24m  10965
                     3m   13449
3m


Trying to set attribute `.obs` of view, copying.


downsampled Matrix
                            age
classification_group age       
A                    3m   45602
B                    18m  10232
                     1m    5890
                     21m   8122
                     24m   8704
                     30m  12654


In [30]:
adata

View of AnnData object with n_obs × n_vars = 110824 × 1061 
    obs: 'FACS.selection', 'age', 'cell', 'cell_ontology_class', 'cell_ontology_id', 'free_annotation', 'method', 'mouse.id', 'sex', 'subtissue', 'tissue', 'n_genes', 'n_counts', 'louvain', 'leiden'
    var: 'n_cells', 'means', 'dispersions', 'dispersions_norm', 'highly_variable'
    uns: 'leiden', 'louvain', 'louvain_colors', 'neighbors', 'pca', 'tissue_colors'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'

In [31]:
senis_facs_tfs_age

,18m,18m_gini,21m,21m_gini,24m,24m_gini,3m,3m_gini
0,Jund,0.038866,Sox9,0.067873,Gpbp1l1,0.019859,Jund,0.065056
1,Klf13,0.016523,Fosl1,0.064039,Gpbp1,0.014834,Fos,0.020596
2,Fos,0.014143,Cebpb,0.057457,Nfia,0.013370,Ybx1,0.019932
3,Ybx1,0.013826,Sox10,0.043444,Ctnnb1,0.013155,Ssbp4,0.014792
4,Hnrnpab,0.011728,Gata3,0.038431,Hnrnpk,0.013019,Ctnnb1,0.013896
...,...,...,...,...,...,...,...,...
208,Nfkb2,0.002751,Hes6,0.000698,Ifi203,0.002577,Atf6,0.002442
209,Atf5,0.002729,Nr4a2,0.000640,Taf1,0.002532,Clock,0.002438
210,Esrra,0.002619,Taf4b,0.000552,Jarid2,0.002521,Rbpj,0.002433
211,Cdx1,0.002613,Lyl1,0.000551,Zfp110,0.002465,Bach2,0.002393


In [ ]:
age = '24m'
sum(senis_droplet_tfs_age[age].isin(senis_facs_tfs_age[age]))/sum(~senis_droplet_tfs_age[age].isna())*100

In [ ]:
sum(senis_facs_tfs_age['3m'].isin(senis_droplet_tfs_age['3m']))/sum(~senis_facs_tfs_age['3m'].isna())*100

In [ ]:
sum(~senis_facs_tfs_age['3m'].isna()),sum(~senis_droplet_tfs_age['3m'].isna())

In [ ]:
sum(senis_facs_tfs_age['3m'].isin(senis_droplet_tfs_age['3m']))

In [ ]:
senis_droplet_tfs_age.to_csv("senis_droplet_tfs_age.csv")

In [34]:
senis_facs_tfs_age.to_csv("senis_facs_tfs_age.csv")

In [ ]:
liverAgeSmallnorm = scRFE (dataMatrix = adataLiver, classOfInterest = 'age', 
                       nEstimators = 10, randomState = 0,  
                  nJobs = -1, oobScore = True, Step = 0.2, Cv = 3)

In [ ]:
liverAgeSmall.to_csv("scRFE_Liver_droplet_age.csv")

In [ ]:
liverAgeSmallnonorm

In [ ]:
liverAgeSmallnorm.to_csv("scRFE_Liver_facs_age.csv")